In [2]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [18]:

plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colors = ['#043ef9', '#eb5334', '#34eb37']
def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(   #Adding candlestick chart
        x=df_plot.open_time, open=df_plot.open, high=df_plot.high, low=df_plot.low, close=df_plot.close,
        line=dict(width=1), opacity=1,
        increasing_line_color='#2EC866', 
        decreasing_line_color='#FF3A4C',
        increasing_fillcolor='#24A06B', 
        decreasing_fillcolor='#CC2E3B',
        )) 
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.open_time,
            y=df_buys[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colors[i], size=12)
        ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.open_time,
            y=df_sells[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colors[i], size=12)
        ))

    fig.add_trace(go.Scatter(
        x=df_buys.open_time,
        y=df_buys.close,
        mode='markers',
        marker=dict(color='#0ec4ce', size=12)
    ))

    fig.add_trace(go.Scatter(
        x=df_sells.open_time,
        y=df_sells.close,
        mode='markers',
        marker=dict(color='#043ef9', size=12)
    ))

    fig.update_layout(width=1100, height=450,
                    margin=dict(l=30, r=30, t=30, b=30),
                    font=dict(color='#e1e1e1'), 
                    paper_bgcolor='#1e1e1e',
                    plot_bgcolor='#1e1e1e')

    fig.update_xaxes(
        gridcolor='#1f292f',
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
        #rangebreaks=[dict(bounds=["sat", "mon"])]
    )
    
    fig.update_yaxes(
        gridcolor='#1f292f',
        showgrid=True
    )
    fig.show()

In [3]:
pair = "BTC_USD"
granularity = "1m"
df_raw = pd.read_csv(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['open_time', 'close_time','volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [1]:
SLOSS = 0.4 #stoploss
TPROFIT = 0.8 #take profit
ENTRY_PRC = 0.1 #entry procentage

def direction(row):
    if row.close > row.open:
        return 1
    else:
        return -1
    
def get_signal(row):
    if row.high_prev > row.high and row.low_prev < row.low:
        return row.DIRECTION_prev
    return 0


def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.high_prev
    elif row.SIGNAL == -1:
        return row.low_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else: 
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else: 
        return 0
    

In [35]:
df = df_raw[['open_time', 'open', 'high', 'low', 'close']].copy()
df['RANGE'] = df.high - df.low
df['high_prev'] = df.high.shift(1)
df['low_prev'] = df.low.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [36]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)


In [37]:
df_plot = df.iloc[200:350]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]


In [11]:
df[df.SIGNAL!=0].head()

,open_time,open,high,low,close,RANGE,high_prev,low_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
5,2025-11-01 00:06:00+00:00,109440.00,109440.00,109433.29,109433.29,6.71,109444.45,109415.29,29.16,-1,-1,-1,109412.374,109426.954,109391.962
17,2025-11-01 00:18:00+00:00,109515.36,109515.37,109502.19,109502.20,13.18,109523.84,109482.91,40.93,-1,1,1,109527.933,109507.468,109556.584
26,2025-11-01 00:27:00+00:00,109539.58,109539.59,109531.93,109531.94,7.66,109541.77,109519.99,21.78,-1,1,1,109543.948,109533.058,109559.194
49,2025-11-01 00:50:00+00:00,109734.86,109749.00,109734.86,109749.00,14.14,109749.12,109725.82,23.30,1,1,1,109751.450,109739.800,109767.760
58,2025-11-01 00:59:00+00:00,109737.37,109737.37,109730.77,109730.78,6.60,109740.00,109724.91,15.09,-1,-1,-1,109723.401,109730.946,109712.838


In [16]:
fig = go.Figure()

fig.add_trace(go.Candlestick(   #Adding candlestick chart
    x=df_plot.open_time, open=df_plot.open, high=df_plot.high, low=df_plot.low, close=df_plot.close,
    line=dict(width=1), opacity=1,
    increasing_line_color='#2EC866', 
    decreasing_line_color='#FF3A4C',
    increasing_fillcolor='#24A06B', 
    decreasing_fillcolor='#CC2E3B',
    )) 
for i in range(0, 3):
    fig.add_trace(go.Scatter(
        x=df_buys.open_time,
        y=df_buys[plot_cols[i]],
        mode='markers',
        marker=dict(color=plot_colors[i], size=12)
    ))
for i in range(0, 3):
    fig.add_trace(go.Scatter(
        x=df_sells.open_time,
        y=df_sells[plot_cols[i]],
        mode='markers',
        marker=dict(color=plot_colors[i], size=12)
    ))

fig.add_trace(go.Scatter(
    x=df_buys.open_time,
    y=df_buys.close,
    mode='markers',
    marker=dict(color='#0ec4ce', size=12)
))

fig.add_trace(go.Scatter(
    x=df_sells.open_time,
    y=df_sells.close,
    mode='markers',
    marker=dict(color='#043ef9', size=12)
))

fig.update_layout(width=1100, height=450,
                  margin=dict(l=30, r=30, t=30, b=30),
                  font=dict(color='#e1e1e1'), 
                  paper_bgcolor='#1e1e1e',
                  plot_bgcolor='#1e1e1e')

fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
    #rangebreaks=[dict(bounds=["sat", "mon"])]
)
 
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()

In [39]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.open_time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    
    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)

    
    def check_entry(self, row):
        if self.direction == 1 and row.close >= self.entry or self.direction == -1 and row.close <= self.entry:
            self.index = row.name
            self.opened = row.open_time
            self.running = True

    
    def update_result(self, row):
        if self.direction == 1:
            if row.close >= self.TP:
                self.result = 2.0
            elif row.close <= self.SL:
                self.result = -1.0
        else:
            if row.close <= self.TP:
                self.result = 2.0
            elif row.close >= self.SL:
                self.result = -1.0
            
        if self.result is not None:
            self.running = False
            self.stopped = row.open_time

In [40]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)

    open_trades = [x for x in open_trades if x.stopped is None] #list comprehention

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))


In [41]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [44]:
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2025-11-01 00:18:00+00:00,1,109527.933,109556.584,109507.468,False,-1.0,2025-11-01 00:30:00+00:00,25,2025-11-01 00:26:00+00:00
1,2025-11-01 00:27:00+00:00,1,109543.948,109559.194,109533.058,False,2.0,2025-11-01 00:44:00+00:00,42,2025-11-01 00:43:00+00:00
2,2025-11-01 00:50:00+00:00,1,109751.450,109767.760,109739.800,False,-1.0,2025-11-01 00:52:00+00:00,50,2025-11-01 00:51:00+00:00
3,2025-11-01 00:59:00+00:00,-1,109723.401,109712.838,109730.946,False,-1.0,2025-11-01 01:20:00+00:00,77,2025-11-01 01:18:00+00:00
4,2025-11-01 01:19:00+00:00,-1,109697.789,109643.182,109736.794,False,2.0,2025-11-01 02:05:00+00:00,121,2025-11-01 02:02:00+00:00


In [45]:
df_trades.result.sum()

np.float64(2451.0)

In [47]:
df[df.SIGNAL!=0].to_pickle("BTC_USD_1m_trades.pkl")